In [1]:
# import cv2
# import matplotlib.pyplot as plt
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# Auto labeling
path = 'C:/Users/Dang Quang/Facial_Recognition/image'
folder_list = os.listdir(path)
folder_list.sort()
labels = {}
for i in range(len(folder_list)):
    labels[i] = folder_list[i]
labels

{0: 'ha', 1: 'quang', 2: 'tam', 3: 'vinh'}

In [3]:
model = tf.keras.models.load_model("C:/Users/Dang Quang/Facial_Recognition/model/face_detect_model.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1

In [8]:
import cv2
from cv2 import cuda
cuda.printCudaDeviceInfo(0)

In [1]:
import cv2, os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

cap = cv2.VideoCapture(0)
# Check camera
if not cap.isOpened():
    raise IOError("Cannot open webcam")



MODEL = 'C:/Users/Dang Quang/Facial_Recognition/yolo/yolov3-face.cfg'
WEIGHT = 'C:/Users/Dang Quang/Facial_Recognition/yolo/yolov3-wider_16000.weights'

net = cv2.dnn.readNetFromDarknet(MODEL, WEIGHT)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

IMG_WIDTH, IMG_HEIGHT = 416, 416
model = tf.keras.models.load_model("C:/Users/Dang Quang/Facial_Recognition/model/face_detect_model.h5")

# Auto label
path = 'C:/Users/Dang Quang/Facial_Recognition/image'
folder_list = os.listdir(path)
folder_list.sort()
labels = {}
for i in range(len(folder_list)):
    labels[i] = folder_list[i]

count = 0

while True:
    ret, frame = cap.read()
    # frame is now the image capture by the webcam (one frame of the video)
    
    
    # Making blob object from original image
    blob = cv2.dnn.blobFromImage(frame, 1/255, (IMG_WIDTH, IMG_HEIGHT), [0,0,0], 1, crop=False)
    # Set model input
    net.setInput(blob)
    # Define the layers that we want to get the outputs from
    output_layers = net.getUnconnectedOutLayersNames()
    # Run 'prediction'
    outs = net.forward(output_layers)
    

    frame_height = frame.shape[0]
    frame_width = frame.shape[1]
    # Scan through all the bounding boxes output from the network and keep only the ones with high confidence scores. Assign the box's class label as the class with the highest score.

    confidences = []
    boxes = []

    # Each frame produces 3 outs corresponding to 3 output layers
    for out in outs:
        # One out has multiple predictions for multiple captured objects.
        for detection in out:
            confidence = detection[-1]
            # Extract position data of face area (only area with high confidence)
            if confidence > 0.5:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)

                # Find the top left point of the bounding box
                topleft_x = int(center_x - width/2)
                topleft_y = int(center_y - height/2)
                confidences.append(float(confidence))
                boxes.append([topleft_x, topleft_y, width, height])

    # Perform non-maximum suppression to eliminate redundant overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)


    result = frame.copy()
    final_boxes = []
    for i in indices:
        i = i[0]
        box = boxes[i]
        final_boxes.append(box)

        # Extract position data
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        
        # Draw bouding box with the above measurements
        cv2.rectangle(result, (left, top), (left + width, top + height), (0,255,0), 2)
        try:
            predict_img = frame[top: top+height, left: left + width]
            
            predict_img = cv2.resize(predict_img, (150, 150))
            img_array = img_to_array(predict_img)
            img_array = np.expand_dims(img_array, axis=0)
            predict_img_processed = preprocess_input(img_array)

            # processing = ImageDataGenerator(preprocessing_function = preprocess_input)
            # predict_img_processed = processing.flow(predict_img, batch_size = 16)

            predictions = model.predict(predict_img_processed)
            print(predictions)
            text = f'{labels[np.argmax(predictions[0])]}_{confidences[i]:.2f}'
            cv2.putText(result, text, (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)

        except Exception as e:
            print(str(e))
    text2 = f'Number of faces detected: {len(indices)}'
    cv2.putText(result, text2, (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
    
    cv2.imshow('face detection', result)


    # cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    # Break when pressing ESC
    if c == 27:
        break
    # elif c == ord('c'):
    #     for i in indices:
    #         i = i[0]
    #         box = boxes[i]
    #         final_boxes.append(box)

    #         # Extract position data
    #         left, top, width, height = box[0], box[1], box[2], box[3]
    #         face = frame[top: top+height, left: left + width]
    #         cv2.imwrite('C:/Users/Dang Quang/Facial_Recognition/image/trieu/trieu'+str(count)+'.jpg', face)
    #         count +=1

cap.release()
cv2.destroyAllWindows()
cv2.imwrite()

[[0.00195989 0.9922179  0.00232777 0.00349437]]
[[5.0643861e-04 9.9861109e-01 5.1804812e-04 3.6442635e-04]]
[[9.6445868e-04 9.9396580e-01 1.7095259e-03 3.3602444e-03]]
[[0.00741553 0.9709343  0.01111568 0.01053455]]
[[0.00191857 0.99012274 0.00291115 0.00504754]]
[[0.00170518 0.99410075 0.00219038 0.00200365]]
[[5.8456371e-04 9.9647212e-01 1.8157050e-03 1.1276845e-03]]
[[0.00157233 0.9942334  0.00185406 0.00234023]]
[[0.00166999 0.9903376  0.0027756  0.00521677]]
[[8.3890802e-04 9.9509573e-01 2.2030342e-03 1.8623263e-03]]
[[0.00198105 0.9905442  0.00240377 0.00507097]]
[[9.8488491e-04 9.9536151e-01 1.5574913e-03 2.0961855e-03]]
[[0.00126783 0.99271387 0.00229289 0.00372546]]
[[8.7977224e-04 9.9475729e-01 1.9619451e-03 2.4009959e-03]]
[[0.00101623 0.99447024 0.00201216 0.00250136]]
[[5.0427607e-04 9.9634886e-01 2.0618599e-03 1.0849624e-03]]
[[6.1748736e-04 9.9698871e-01 1.0705999e-03 1.3231282e-03]]
[[0.00122977 0.99319446 0.0017138  0.00386186]]
[[7.0120284e-04 9.9653804e-01 9.7039348e

error: OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - imwrite() missing required argument 'filename' (pos 1)
>  - imwrite() missing required argument 'filename' (pos 1)
